# Ising Model

![title](figs/ising_model.png)

In this project you will use the Metropolis algorithm to investigate ferromagnetism using a simplified model of the spin‑spin interactions known as the [Ising model](https://en.wikipedia.org/wiki/Ising_model). In the Ising model we treat the magnet as a set of spins on fixed lattice sites in two dimensions. The remarkably simple model predicts many features of real ferromagnetic materials such as second order phase transitions in temperature ([Curie point](https://en.wikipedia.org/wiki/Curie_temperature)).

Suppose we have an $N$ by $N$ 2D lattice, for a total of $N^2$ sites. The energy of the system is written
$$
E = - \sum_{(ij)}{s_i s_j}
$$
where $(ij)$ is the sum over nearest neighbour pairs of spins, four total in the 2D case. 

In order to calculate an observable of the system such as the magnetisation (average spin value $ M = \frac{1}{N^2} \sum s_i $), we need to calculate the [canonical ensemble](https://en.wikipedia.org/wiki/Canonical_ensemble) of the system at a given temperature, and then take the average $\langle M \rangle = \overline{M}$ over many microstates. Therefore, we need an algorithm which will generate lattice configurations according to the [Boltzman distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution) at temperature $T$
$$
P(\text{Lattice} ; E) = \frac{1}{Z} e^{-E / T}
$$

Don't worry if the physics behind this is unfamiliar, its really not important. The main idea here is that we have a high dimensional function $E$ (energy) of $N^2$ parameters (spin). And we want to calculate (or estimate) the probability distribution over these parameters according to $P(E)$. 

## Metropolis Algorithm

The [Metropolis algorithm](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) is a Markov-Chain Monte-Carlo method for obtaining a sequence of random samples that is guaranteed to converge to a target distribution. In the last exercise using pure Monte-Carlo, each sample was generated independently with $x,y \in [0,1]$. 

With the Metropolis algorithm, we perform local (small) updates on an initial sample and then choose to probabilistically accept or reject the new sample. If we accept the sample, we add it to our chain and then repeat the process. The intuition with this algorithm is that most of the chain should have lattices with low energy, but we still want some configurations with a high energy to match the desired Boltzmann distribution.

The metropolis algorithm for evolving the lattice microstates is as follows
1. Select a spin. Calculate $ \Delta E $ to flip it.
3. Select a random number $p \in [0,1] $.
4. If $ \exp(- \Delta E / T ) > p $, flip the spin.

This process is repeated for all $N^2$ spins, after which constitutes a single $\text{Monte Carlo time step}$. The newly calculated lattice is then added to a list of Markov-Chain samples. 

To calculate an observable such as the magnetisation, we simply take the mean value of the magnetisation for each sample in the list.
$$
\langle M \rangle = \frac{1}{n_{\text{samples}}} \sum_{t=0}^{\text{samples}} M_t 
$$
$$
M_t = \frac{1}{N^2} \sum_i s_i 
$$

Here is a plot of a Markov-Chain sequence where we calculate the magnetisation of 1000 samples and take the mean of the entire chain.
![mags](figs/magnetisation.png)

## Phase Diagram

Our main goal is to observe a [phase transition](https://en.wikipedia.org/wiki/Phase_transition) in the Ising model. To do this we have to calculate the (absolute) mean magnetisation $| \langle M \rangle |$ for a range of different temperatures. Plotting these on a graph we hope to see a discontinuity where the magnetisation suddenly drops to 0. 

![phase_transition](figs/phase_transition.png)

For reference we have also plotted the [analytical value](https://en.wikipedia.org/wiki/Ising_model#Two_dimensions) for the critical temperature $ T_c = 2/\ln(1+\sqrt(2)) \approx 2.26918 $.

It is your task to implement this algorithm and create a similar phase diagram. Start with a small $N=10$ lattice and see how the phase diagram compares against larger lattices. Try to generate at least $1,000$ samples for each temperature.

## Extension (Optimisations)

The Metropolis algorithm for the Ising model can be computed thousands of times faster using an optimised algorithm that exploits the symmetry of the model. The 'checkerboard' algorithm is described in this [paper](https://arxiv.org/pdf/1906.06297.pdf) and can be implemented in python with not too much extension. Try to use np.roll or jax.roll for GPU acceleration and measure the performance difference. 

In [1]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

CRITICAL_TEMP = 2.26918